In [24]:
import cv2
import easyocr
import json
from PIL import Image, ImageDraw
import re

# EasyOCR reader 객체 생성 (한글 및 영어 지원)
reader = easyocr.Reader(['ko', 'en'], gpu=False)

def process_image(img):
    # 이미지 읽기
    image = cv2.imread(img)
    if image is None:
        raise Exception(f"이미지를 읽을 수 없습니다: {img}")
    
    # 이미지에서 텍스트 추출
    results = reader.readtext(image)
    
    # 추출된 텍스트를 저장할 리스트
    extracted_text = []
    
    # 파란색 상자를 그리기 위한 PIL 이미지로 변환
    img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img)
    
    # 각 텍스트 주위에 상자를 그리고 텍스트를 저장
    for detection in results:
        bbox = detection[0]
        text = detection[1]
        extracted_text.append(text)
        
        # 상자 좌표
        x_min, y_min = int(bbox[0][0]), int(bbox[0][1])
        x_max, y_max = int(bbox[2][0]), int(bbox[2][1])
        
        # 상자 그리기
        draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=2)
        draw.text((x_min, y_min - 20), text, fill="blue")
    
    # 정보 추출 함수들
    def extract_business_numbers(text_list):
        business_number_pattern = re.compile(r'\b\d{3}-?\d{2}-?\d{5}\b')
        return [match for text in text_list for match in business_number_pattern.findall(text)]
    
    def extract_store_names(text_list):
        # 오타를 포함한 키워드 패턴
        store_name_pattern = re.compile(r'(상호명|회사명|업체명|가맹점명|가맣점명|상오|[상싱성][호오]|[회훼]사)\s*[:;：]?\s*([^\s)]+(?:\s*\S*)*?[점]\s*?\S*)')
        return [match[1] for text in text_list for match in store_name_pattern.findall(text)]

    def extract_transaction_date(json_data):
        # 다양한 날짜/시간 형식을 허용하는 정규식
        date_pattern = re.compile(
            r'([거기][래레][일닐][시자]|[결겔]제[일닐]시|거[래레]일|[결겔]제날짜|날짜)\s*[:;：]?\s*(\d{2,4}[-/]\d{2}[-/]\d{2})\s*([0-2]?\d)?[,:]?\s*([0-5]?\d)?[,:]?\s*([0-5]?\d)?'
        )
    
        extracted_dates = []
    
        for text in json_data:  # json_data가 문자열 목록이라고 가정
            matches = date_pattern.findall(text)
            
            for match in matches:
                if match[2] and match[3] and match[4]:  # 시간이 모두 존재할 경우
                    extracted_dates.append(f"{match[1]} {match[2]}:{match[3]}:{match[4]}")
                else:
                    extracted_dates.append(match[1])  # 시간 정보가 없을 경우
    
        return extracted_dates
        
        
    def extract_transaction_date(json_data):
        # 날짜와 시간 형식, 또는 날짜만 있는 경우를 모두 허용하는 정규식
        date_pattern = re.compile(
            r'([거기][래레][일닐][시자]?|[결겔]제[일닐]시|거[래레]일|[결겔]제날짜|날짜)?\s*[:;：]?\s*(\d{4}[-/]\d{2}[-/]\d{2})\s*([0-2]?\d)?[,:]?\s*([0-5]?\d)?[,:]?\s*([0-5]?\d)?'
        )

        extracted_dates = []
    
        for text in json_data:  # json_data가 문자열 목록이라고 가정
            matches = date_pattern.findall(text)
        
            for match in matches:
                date_part = match[1] if match[1] else match[0]  # 날짜 부분만 추출
                if match[2] and match[3] and match[4]:  # 시간이 모두 존재할 경우
                    extracted_dates.append(f"{date_part} {match[2]}:{match[3]}:{match[4]}")
                else:
                    extracted_dates.append(date_part)  # 시간 정보가 없을 경우 날짜만 추출

        return extracted_dates

    
    # def extract_transaction_date(json_data):
    #     # 다양한 날짜/시간 형식을 허용하는 정규식
    #     date_pattern = re.compile(
    #         r'([거기][래레][일닐][시자]?|[결겔]제[일닐]시|거[래레]일|[결겔]제날짜|날짜)?\s*[:;：]?\s*'
    #         r'((\d{2,4}[-/]\d{2}[-/]\d{2})|(\d{2}[-/]\d{2}[-/]\d{2,4}))\s*([0-2]?\d)?[,:]?\s*([0-5]?\d)?[,:]?\s*([0-5]?\d)?'
    #     )

    #     extracted_dates = []
    
    #     for text in json_data:  # json_data가 문자열 목록이라고 가정
    #         matches = date_pattern.findall(text)
        
    #         for match in matches:
    #             date_part = match[1] if match[1] else match[0]  # 날짜 부분만 추출
            
    #             # 날짜 형식 변환: MM-DD-YYYY 또는 MM/DD/YYYY 형태를 YYYY-MM-DD로 표준화
    #             if re.match(r'\d{2}[-/]\d{2}[-/]\d{4}', date_part):
    #                 day, month, year = date_part.split(date_part[2])
    #                 date_part = f"{year}-{month}-{day}"
            
    #             if match[4] and match[5] and match[6]:  # 시간이 모두 존재할 경우
    #                 extracted_dates.append(f"{date_part} {match[4]}:{match[5]}:{match[6]}")
    #             else:
    #                 extracted_dates.append(date_part)  # 시간 정보가 없을 경우 날짜만 추출

    #     return extracted_dates
    
    
    def extract_total_price(text_list):
    # '총' 또는 '합계' 뒤에 오는 금액을 추출
        total_price_pattern = re.compile(
            r'(총|합 계|합계|합계금액|Subtotal|total)\s*가격?\s*[:;：]?\s*([₩$€]?\s?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'
        )
        return [match[1] for text in text_list for match in total_price_pattern.findall(text)]

    
    # 정보 추출
    info_dict = {
        "사업자번호": extract_business_numbers(extracted_text),
        "가맹점명": extract_store_names(extracted_text),
        "거래일시": extract_transaction_date(extracted_text),
        "총 가격": extract_total_price(extracted_text)
    }
    
    return info_dict

# 이미지 처리 및 정보 추출
a = process_image('bill6.jpeg')
print(json.dumps(a, ensure_ascii=False, indent=4))


Using CPU. Note: This module is much faster with a GPU.


{
    "사업자번호": [],
    "가맹점명": [],
    "거래일시": [
        "2018/01/30"
    ],
    "총 가격": []
}
